In [1]:
# https://github.com/znxlwm/tensorflow-MNIST-GAN-DCGAN
# https://github.com/znxlwm/tensorflow-MNIST-GAN-DCGAN/blob/master/tensorflow_MNIST_DCGAN.py

import os, time, pickle, itertools #, imageio
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


def lrelu(x, th=0.2):
    return tf.matmul(th * x, x)
    
    
def generator(x, is_train=True, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        
        # 1st hidden layer
        conv1 = tf.layers.conv2d_transpose(x, 1024, [4, 4], strides=(1, 1), padding='valid')
        lrelu1 = lrelu(tf.layers.batch_normalization(conv1, training=is_train), 0.2)
        
        # 2nd hidden layer
        conv2 = tf.layers.conv2d_transpose(lrelu1, 512, [4, 4], strides=(2, 2), padding='same')
        lrelu2 = lrelu(tf.layers.batch_normalization(conv2, training=is_train), 0.2)
        
        # 3rd hidden layer
        conv3 = tf.layers.conv2d_transpose(lrelu2, 256, [4, 4], strides=(2, 2), padding='same')
        lrelu3 = lrelu(tf.layers.batch_normalization(conv3, training=is_train), 0.2)
        
        # 4th hidden layer
        conv4 = tf.layers.conv2d_transpose(lrelu3, 128, [4, 4], strides=(2, 2), padding='same')
        lrelu4 = lrelu(tf.layers.batch_normalization(conv4, training=is_train), 0.2)
        
        # output layer
        conv5 = tf.layers.conv2d_transpose(lrelu4, 1, [4, 4], strides=(2, 2), padding='same')
        output = tf.nn.tanh(conv5)
        
        return output
    

def discriminator(x, is_train=True, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        
        # 1st hidden layer
        conv1 = tf.layers.conv2d(x, 128, [4, 4], strides=(2, 2), padding='same')
        lrelu1 = lrelu(conv1, 0.2)
        
        # 2nd hidden layer
        conv2 = tf.layers.conv2d(lrelu1, 256, [4, 4], strides=(2, 2), padding='same')
        lrelu2 = lrelu(tf.layers.batch_normalization(conv2, training=is_train), 0.2)
        
        # 3rd hidden layer
        conv3 = tf.layers.conv2d(lrelu2, 512, [4, 4], strides=(2, 2), padding='same')
        lrelu3 = lrelu(tf.layers.batch_normalization(conv3, training=is_train), 0.2)
        
        # 4th hidden layer
        conv4 = tf.layers.conv2d(lrelu3, 1024, [4, 4], strides=(2, 2), padding='same')
        lrelu4 = lrelu(tf.layers.batch_normalization(conv4, training=is_train), 0.2)
        
        # output layer
        conv5 = tf.layers.conv2d(lrelu4, 1, [4, 4], strides=(1, 1), padding='valid')
        output = tf.nn.sigmoid(conv5)
        
        return output, conv5
        
        
fixe_z_ = np.random.normal(0, 1, (25, 1, 1, 100))
def show_result(num_epoch, show=False, save=False, path='result.png'):
    test_images = sess.run(G_z, feed_dict={z:fixed_z_, is_train:False})
    
    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)
        
    for k in range(5*5):
        i = k // size_figure_grid
        j = k % size_figure_grid
        ax[i, j].cla()
        ax[i, j].imshow(np.reshape(test_images[k], (64, 64)), cmap='gray')
        
    if save:
        plt.savefig(path)
    
    if show:
        plt.show()
    else:
        plt.close()


def show_train_hist(hist, show=False, save=False, path='Train_hist.png'):
    x = range(len(hist['D_losses']))
    
    y1 = hist['D_losses']
    y2 = hist['G_losses']
    
    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')
    
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()
    
    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()


batch_size = 100
learning_rate = 0.0002
epochs = 20

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True, reshape=[])

# variables : input
x = tf.placeholder(tf.float32, shape=(None, 64, 64, 1))
z = tf.placeholder(tf.float32, shape=(None, 1, 1, 100))
is_train = tf.placeholder(tf.bool)

# networks : generator
G_z = generator(z, is_train)

# networks : discriminator
D_real, D_real_logits = discriminator(x, is_train)
D_fake, D_fake_logits = discriminator(G_z, is_train, reuse=True)

# loss for each network
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones([batch_size, 1, 1, 1])))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logtis(logits=D_fake_logits, lables=tf.zeros([batch_size, 1, 1, 1])))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones([batch_size, 1, 1, 1])))

# trainable variables for each network
T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
G_vars = [var for var in T_vars if var.name.startswith('generator')]

# optimizer for each network
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    D_optim = tf.train.AdamOptimizer(learning_rate, beta1=0.5).minimize(D_loss, var_list=D_vars)
    G_optim = tf.train.AdamOptimizer(learning_rate, beta1=0.5).minimize(G_loss, var_list=G_vars)
    
# open session and initialize all variables
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# MNIST resize and normalization
train_set = tf.image.resize_images(mnist.train.images, [64, 64]).eval()
train_set = (train_set - 0.5) / 0.5   # normalization range: -1 ~ 1

# results save folder
root = "MNIST_DCGAN_results/"
model = "MNIST_DCGAN_"
if not os.path.isdir(root):
    os.mkdir(root)
if not os.path.isdir(root + 'Fixed_results'):
    os.mkdir(root + 'Fixed_results')
    
train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []

# training loop
np.random.seed(int(time.time()))
start_time = time.time()
for e in range(epochs):
    G_losses = []
    G_losses = []
    epoch_start_time = time.time()
    
    batches = mnist.train.num_examples // batch_size
    for batch_i in range(batches):
        
        # update discriminator
        x_ = train_set[batch_i*batch_size:(batch_i+1)*batch_size]
        z_ = np.random.normal(0, 1, (batch_size, 1, 1, 100))

        d_loss_, _ = sess.run([D_loss, D_optim], feed_dict={x:x_, z:z_, is_train: True})
        D_losses.append(d_loss_)
        
        # update generator
        z_ = np.random.normal(0, 1, (batch_size, 1, 1, 100))
        g_loss_, _ = sess.run([G_loss, G_optim], feed_dict={z:z_, x:x_, is_train: True})
        G_losses.append(g_loss_)
        
    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time
    print("Epoch: {}/{}, {:.2f}s/epoch, d_loss mean: {:.5f}, g_loss mean: {:.5f}"
          .format(e, epochs, per_epoch_ptime, np.mean(D_losses), np.mean(G_losses)))
    
    fixed_p = root + 'Fixed_results/' + model + str(e + 1) + '.png'
    show_result((e + 1), save=True, path=fixed_p)
    
    train_hist['D_losses'].append(np.mean(D_losses))
    train_hist['G_losses'].append(np.mean(G_losses))
    train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

end_time = time.time()
total_ptime = end_time - start_time
train_hist['total_ptime'].append(total_ptime)

print("Avg per epoch ptime: {}, {} Epochs have taken {:.2f}"
      .format(np.mean(train_hist['per_epoch_ptimes']), epochs, total_ptime))
print('Training Finished!')
with open(root + model + 'train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)
    
show_train_hist(train_hist, save=True, path=root + model + 'train_hist.png')

images = []
for e in range(epochs):
    img_name = root + 'Fixed_results/' + model + str(e + 1) + '.png'
    images.append(imageio.imread(img_name))
imageio.mimsave(root + model + 'generation_animation.gif', images, fps=5)

sess.close()


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


ValueError: Dimensions must be equal, but are 1024 and 4 for 'generator/MatMul' (op: 'BatchMatMul') with input shapes: [?,4,4,1024], [?,4,4,1024].